In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
url = "https://github.com/swarj/MLProject/raw/main/apple_share_price.csv" #get the dataset
raw_df = pd.read_csv(url)
raw_df = raw_df.loc[::-1] #reverse order of rows to chronologically make it go from past --> present
raw_df = raw_df.assign(Time=range(len(raw_df)))
df = raw_df.reindex(['Time','Date', 'High', 'Low', 'Volume', 'Open', 'Close'], axis = 1) #make target variable last column, if we want close instead of open we can switch it
#we can try to predict the close value for the following day, need to add target close value  
df['TargetNextClose'] = df['Close'].shift(-1)
df.drop(df.tail(1).index,inplace=True) # drop last rows
df.style

,Time,Date,High,Low,Volume,Open,Close,TargetNextClose
1663,0,3-Jan-11,47.180000,46.410000,111280407,46.520000,47.080000,47.330000
1662,1,4-Jan-11,47.500000,46.880000,77337001,47.490000,47.330000,47.710000
1661,2,5-Jan-11,47.760000,47.070000,63879193,47.080000,47.710000,47.680000
1660,3,6-Jan-11,47.890000,47.560000,75106626,47.820000,47.680000,48.020000
1659,4,7-Jan-11,48.050000,47.410000,77982212,47.710000,48.020000,48.920000
1658,5,10-Jan-11,49.030000,48.170000,112139482,48.400000,48.920000,48.810000
1657,6,11-Jan-11,49.280000,48.500000,111019993,49.270000,48.810000,49.200000
1656,7,12-Jan-11,49.200000,48.860000,75644310,49.040000,49.200000,49.380000
1655,8,13-Jan-11,49.520000,49.120000,74536182,49.310000,49.380000,49.780000
1654,9,14-Jan-11,49.780000,49.210000,77209748,49.410000,49.780000,48.660000
